In [1]:
import os
import sys
import math
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib import ticker
import datetime as dt

In [5]:
# Constant
g         = 9.8086# (m s-2) gravity 
Pi        = math.pi
rho       = 1000.



Csw =      # C%CSW         determine decrease rate of standard deviation of canopy vertical velocity
Ctl =      # C%CTL (K s-1) relates to temperature under canopy to the friction velocity and canopy height

# input parameters
LAI       =  # (m2 m-2) leaf area index 
Rwv       = *        # gas constand for water vapor
hyds =
bch  = 

z         =  # height above the soil surface of U_x
theta_sat = #ssat, saturated water content
theta_r   = #watr, residual water content

# input data
v         =                   # (m2 s-1)the kinematic viscosity of air
Phi_1     =         # soil matric potential in the first soil layer
Tsrf      = function(K) # soil surface temperature
qsat      = function(Tsrf)    # the saturated specific humidity at the surface temperature
qa        =  # (kg kg-1) specific humidity of the canopy air ;  the specific humidity of the atmosphere
Uair      = #(m s-1) observed wind speed away from the surface and canopy 
Ux        = # friction velocity
U_x       = Ux*exp(Cv*(1.-z/h)) # (m s-1) canopy friction velocity 
Cv        = # extinction coefficient for wind speed in the canopy
theta     = # wb






IndentationError: unindent does not match any outer indentation level (<tokenize>, line 58)

In [ ]:
# CALCULATE Rg    

L  = LAI # canopy%rghlai (m) LAI over the whole grid cell, it is not Monin-Obhukov length,Decker 2017 made a mistake  
# canopy%rghlai = veg%vlai * rough%hruff / MAX( 0.01, veg%hc )

h  = 35. # rough%hruff   (m) canopy height for evergreen broadleaf
#rough%hruff = MAX( 1.e-6,veg%hc - 1.2*ssnow%snowd/MAX(ssnow%ssdnn,100.)), canopy height above snow level

d  = (1.0-(1.0-math.exp(-1.0*math.sqrt(15.*max((L*0.5),0.0005))))/(math.sqrt(15.*max((L*0.5),0.0005))))*h
         # rough%disp    (m) the displacement height of the canopy  
# xx = SQRT( 15.0 * MAX( ( canopy%rghlai * 0.5 ), 0.0005 ) )
# dh = 1.0-(1.0 - EXP(-xx ))/xx, eq.8 Raupach 1994, BLM, vol 71, p211-216:
# rough%disp = dh * rough%hruff, calculate zero-plane displacement:

Z0s = 0.01*LAI + 0.02*U_x**2/g # rough%z0soilsn (m) soil surface roughness height, turbulent airflow's length scale
#rough%z0soil   = 0.01*min(1.0,canopy%vlaiw) + 0.02*min(canopy%us**2/C%GRAV,1.0)
#rough%z0soilsn = max(1.e-7,rough%z0soil) 

a_3 = 1.25 # C%A33         inertial sublayer sw/us, given by the ratio of the standard deviation of vertical
           #               velocity to the friction velocity in the inertial sublayer above the canopy

fsp = 1./ max(2/3*h/d, 1.) # sparseness factor (see p.7 in CABLE guide 2016), =1 for bare ground; > 1 when LAI > 1
# The larger LAI -> the larger d (d always smaller than h) -> the larger fsp, but fsp cannot larger than 1
# that is to say, when LAI lower than 1 -> fsp lower than 1 -> rg will increase -> harder to evaporate

rg  = math.log(d/Z0s)*(math.exp(2.*Csw*L)-math.exp((2.*Csw*L) *(1 -d/h)))/(2.*a_3**2*fsp*Ctl*Csw*L)
      # the resistance for both the sensible and latent heat fluxes from the soil to the canopy air 
# rough%rt0us  = (MAX((2./3.)*rough%hruff/rough%disp,1.0))*LOG(rough%disp/rough%z0soilsn)*
#                (EXP(2*C%CSW*canopy%rghlai)-EXP(2*C%CSW*canopy%rghlai*(1-rough%disp/rough%hruff)))/
#                (C%A33**2 * C%CTL * 2 * C%CSW * canopy%rghlai)
# ssnow%rtsoil(j) = max(rt_min,rough%rt0us) / canopy%us

In [ ]:
# calculate rsv

theta_srf = (theta -theta_r)/(theta_sat - theta_r) # relative wetness at the surface
qsrf = qsat*math.exp((Phi_1*g)/(Rwv*Tsrf))  # (kg kg-1) specific humidity at the soil surface 

gamma = 0.0000173 # lm=1.73e-5, converts units
K     = 0.001*hyds*theta_srf**(2.*bch+3.) #(m s-1) hydraulic conductivity of the soil at the surface
# hk_zero(i) = max(0.001*soil%hyds_vec(i,1)*(min(max(rel_s(i),0.001_r_2),1._r_2)**(2._r_2*soil%bch_vec(i,1)+3._r_2) ),1e-12)
rsv   = gamma/(4.*K)
# rsv = lm/ (4.0*hk_zero(i))

In [ ]:
# calculate rBL
sigma     = 0.071 # (Jm-2) surface tension of liquid water 
Phi_sat   =  # abs(soil%sucs_vec(:,1)/1000. (m) the soil matric potential at saturation 
alpha     = 0.3/U_x*(Uair - 1) # time spectrum parameter
eddy_shape(i) = 0.3*met%ua(i)/ max(1.0e-4, max(1.0e-3,canopy%us(i))* exp(-rough%coexp(i) *
                (1.0-canopy%sublayer_dz(i) / max(1e-2,rough%hruff(i))) ) )
int_eddy_shape(i) = floor(eddy_shape(i))

Chi  = 1
for n in np.arange(1,alpha,1):
    Chi  = Chi*(2.*alpha+1.)*(2.*(alpha -n)+1)
 ??? Gamma(alpha +1 )
G    = 2.2 * math.sqrt(112.* Pi)/Gamma(alpha +1)/ (2**(alpha + 1)*math.sqrt(alpha +1)) * Chi
       # dimensionless coefficient dependent on eddy residence time spectrum parameter a (-)
#eddy_mod(i) = 2.2*sqrt(112.0*(pi_r_2)) / (2.0**(eddy_shape(i)+1.0) * sqrt(eddy_shape(i)+1.0))

delta  = G*v/U_x # depth of the diffusion dominated region at the soil-air interface,
                        # within delta molecular diffusion dominates transport 
canopy%sublayer_dz(i) = min(0.05, max(eddy_mod(i) * air%visc(i) / max(1.0e-3,canopy%us(i))*
                     exp(-rough%coexp(i)*(1.0-canopy%sublayer_dz(i)/max(1e-2,rough%hruff(i)))),1e-7)  )

if (int_eddy_shape(i) .gt. 0) then
   eddy_mod(i) = eddy_mod(i) / my_gamma(eddy_shape(i)+1.0) * (2.0*eddy_shape(i)+1.0)
   do k=1,int_eddy_shape(i)
      eddy_mod(i) = eddy_mod(i) * (2.0*(eddy_shape(i) - k) + 1.0)
   end do
end if

R     = 2.*sigma/(rho*g*Phi_sat) # pore radius, using Young-Laplace equation
# pore_radius = 0.148/(1000.0*9.81*abs(soil%sucs_vec(:,1))/1000.0) 

F     = 1./(Pi*math.sqrt(theta_srf))*(math.sqrt(Pi/(4.*theta_srf))-1) 
        # soil_moisture_mod, surface wetness dependency model from Schlunder (1988), 
        # the asymptotic solution to constant rate diffusion from a partly wetted surface 
# soil_moisture_mod(i) = 1.0/(pi_r_2)/sqrt(wb_liq(i))* ( sqrt((pi_r_2)/(4.0*wb_liq(i)))-1.0)

D     = 0.0000282 # rt_Dff=2.5e-5 (m2 s-1), diffusion of wtare vapor

rBL   = (delta + R*math.sqrt(Pi)*F)/D
# rBL = (canopy%sublayer_dz(i)+ pore_size(i) * soil_moisture_mod(i)) / rt_Dff



In [ ]:
# calculate Esoil

pres =  # met%pmb * 100.0
tvair=
! molar volume (m^3/mol)
air%volm = C%RGAS * (met%tvair) / (100.0 * met%pmb)

! saturation specific humidity
air%qsat = (C%RMH2O / C%RMAIR) * es / met%pmb
! air kinematic viscosity (m^2/s)
air%visc = 1e-5 * MAX(1.0, 1.35 + 0.0092 * (met%tvair - C%TFRZ) )

rhoa =  min(1.3, 0.02897*pres/(8.3143* tvair))  # (kg m-3)density of dry air 
# air%rho = MIN(1.3, C%RMAIR * (met%pmb * 100.0 / (C%RGAS * (met%tvair))))
# rmair  = 0.02897, molecular wt: dry air (kg/mol)
# rgas   = 8.3143,  universal gas const  (J/mol/K)

REAL, INTENT(IN), DIMENSION(mp) ::  tair,   ! air temperature 
                                    pmb  ! pressure PMB (mb)
REAL, INTENT(OUT), DIMENSION(mp) ::  var ! result; sat sp humidity
INTEGER :: j
DO j=1,mp
   var(j) = (C%RMH2o/C%rmair) * (C%TETENA*EXP(C%TETENB*tair(j)/(C%TETENC+tair(j)))) / pmb(j)
ENDDO

Lv     = 2501400. # air%rlam= C%HL (J kg-1) latent heat of vaporization 

qsrf - qa =  
# dq_unsat = ssnow%rh_srf*ssnow%qstss - met%qvair

unsat_wb(i) =  (ssnow%wb(i,1) - ssnow%ssat_hys(i,1)*&
          min(0.95,max(0.0,ssnow%satfrac(i))))/(1.0 - min(0.95,max(0.0,ssnow%satfrac(i))))
unsat_wb(i) = max(ssnow%watr_hys(i,1)+1e-2, min(ssnow%ssat_hys(i,1), unsat_wb(i) ) )
unsat_smp(i) = sign(ssnow%sucs_hys(i,1),-1.0) * min(1.0, &
             (max(0.01, (unsat_wb(i)-ssnow%watr_hys(i,1))/(ssnow%ssat_hys(i,1)-&
             ssnow%watr_hys(i,1)) ) )** (-soil%bch_vec(i,1)) )
unsat_smp(i) = max(-1.0e4,unsat_smp(i) )/m2mm !m
ssnow%rh_srf(i) = max(0.,min(1., exp(9.81*unsat_smp(i)/(ssnow%tgg(i,1)*461.4)) ) )


Esoil = rhoa * Lv * (qsrf - qa)/(rg + Z0s/delta*(rsv+rBL))
# ssnowpotev =air%rho * air%rlam * dq /ssnow%rtsoil
 rough%z0soil(i)/canopy%sublayer_dz(i)
    
